In [1]:
import geopandas as gpd
import pandas as pd
import os
import getpass
import matplotlib.pyplot as plt
import re
import ipyleaflet as ipy 
import json
import ipywidgets

In [2]:
wd = os.getcwd()
wd = wd[:wd.find('notebooks')]

In [3]:
#landuse = gpd.read_file(wd+\"data//")
acl = gpd.read_file(wd+"data/fulljurisdiction/austincitylimitsgj.geojson")
cbg = gpd.read_file(wd+"data/censusblockgroups/cbggj.geojson")
pop = pd.read_excel(wd+"data/censustables/ACS_17_5yr_totalpop_allcounties.xlsx")
race = pd.read_excel(wd+"data/censustables/ACS_17_5yr_race_includinglatinx.xlsx")
halfbuff = gpd.read_file(wd+"data/coaparks_buffer/halfbuffgj.geojson")
quarbuff = gpd.read_file(wd+"data/coaparks_buffer/quarterbuffgj.geojson")
varibuff = gpd.read_file(wd+"data/coaparks_buffer/varibuffgj.geojson")
parks = gpd.read_file(wd+"data/coaparks/parkboundaries.geojson")
#parks = gpd.read_file("https://data.austintexas.gov/api/geospatial/8f2b-a4q5?method=export&format=GeoJSON")

In [7]:
race

,Id,Id2,Estimate; Total:,Estimate; Not Hispanic or Latino:,Estimate; Not Hispanic or Latino: - White alone,Estimate; Not Hispanic or Latino: - Black or African American alone,Estimate; Not Hispanic or Latino: - American Indian and Alaska Native alone,Estimate; Not Hispanic or Latino: - Asian alone,Estimate; Not Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone,Estimate; Not Hispanic or Latino: - Some other race alone,Estimate; Not Hispanic or Latino: - Two or more races:,Estimate; Hispanic or Latino:,Sum of Non-Hispanic and Hispanic (Total Pop)
0,1500000US482090101001,482090101001,1197,863,769,12,0,65,0,0,17,334,1197
1,1500000US482090101002,482090101002,1174,839,762,63,0,0,0,0,14,335,1174
2,1500000US482090102001,482090102001,870,672,627,42,0,0,0,0,3,198,870
3,1500000US482090102002,482090102002,3914,2665,1994,391,17,146,0,0,117,1249,3914
4,1500000US482090102003,482090102003,1207,769,482,190,0,52,0,0,45,438,1207
...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,1500000US484910216013,484910216013,2590,1794,1619,100,0,11,0,8,56,796,2590
899,1500000US484910216021,484910216021,1956,1179,995,160,0,13,0,0,11,777,1956
900,1500000US484910216022,484910216022,1111,1000,938,37,0,4,0,0,21,111,1111
901,1500000US484910216031,484910216031,667,459,448,0,0,11,0,0,0,208,667


In [6]:
cbg['GEOID10'].str.slice(start=0, stop=12).equals(pop['Id2'])

False

In [5]:
halfbuf_data = ipy.GeoData(geo_dataframe = halfbuff,
                       style={'color': 'black', 'fillColor': '#ffaa00', 'opacity':0.5, 'weight':1, 'dashArray':'2', 'fillOpacity':0.6},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Half Mile Buffer')

varibuf_data = ipy.GeoData(geo_dataframe = varibuff,
                       style={'color': 'black', 'fillColor': '#fcf800', 'opacity':0.5, 'weight':1, 'dashArray':'2', 'fillOpacity':0.6},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Variable Mile Buffer')

#acl_data = ipy.GeoData(geo_dataframe = acl,
                      # style={'color': 'white', 'opacity':0.05, 'weight':5, 'dashArray':'2', 'fillOpacity':0.6},
                       #name = 'Austin City Limits')
#m.add_layer(acl_data)

In [6]:
m = ipy.Map(center=(30.26,-97.74), zoom = 10, basemap= ipy.basemaps.Esri.WorldTopoMap)

def hover_handler(event=None, id=None, properties=None, feature=None):
    label_name.value='Park: '+ str(properties['LOCATION_N'])
    label_parks.value = 'Park acreage: ' + str(properties['ASSET_SIZE'])
    label_buff.value = 'Area within park: '+ str(dropdown.value)

# Not in use anymore..pending deletion
def hover_handler2(event=None, id=None, properties=None, feature=None):
    label_name.value='Park: '+ str(properties['LOCATION_N'])
    label_parks.value = 'Park acreage: ' + str(properties['ASSET_SIZE'])
    

def on_dropdown_change(change):
    if change.new == 'Quarter Mile Buffer':
        if len(layer_group.layers) > 1:
            layer_group.remove_layer(layer_group.layers[1])
        layer_group.add_layer(quarbuf_data)
    if change.new == 'Half Mile Buffer':
        if len(layer_group.layers) > 1:
            layer_group.remove_layer(layer_group.layers[1])
        layer_group.add_layer(halfbuf_data)
    if change.new == 'Varied Buffers':
        if len(layer_group.layers) > 1:
            layer_group.remove_layer(layer_group.layers[1])
        layer_group.add_layer(varibuf_data)

        

quarbuf_data = ipy.GeoData(geo_dataframe = quarbuff,
                       style={'color': 'black', 'fillColor': '#eb0c0c', 'opacity':0.5, 'weight':1, 'dashArray':'2', 'fillOpacity':0.6},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Quarter Mile Buffer')

park_data = ipy.GeoData(geo_dataframe = parks,
                       style={'color': 'black', 'fillColor': '#5c8000', 'opacity':1, 'weight':1, 'dashArray':'2', 'fillOpacity':0.6},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Parks'
                       )


label_parks = ipywidgets.Label(layout=ipywidgets.Layout(width='100%'))
label_name = ipywidgets.Label(layout=ipywidgets.Layout(width='100%'))
label_buff = ipywidgets.Label(layout=ipywidgets.Layout(width='100%'))


dropdown = ipywidgets.Dropdown(options=['Quarter Mile Buffer', 'Half Mile Buffer', 'Varied Buffers'],
                    value='Quarter Mile Buffer', 
                    description='Radius within Parks',
                    disabled=False)
display(dropdown)
dropdown.observe(on_dropdown_change)

park_data.on_hover(hover_handler)
#acl_data.on_hover(hover_handler_acl)
#m.add_layer(acl_data)
layer_group = ipy.LayerGroup(layers=(park_data, quarbuf_data))

m.add_layer(layer_group)
m.add_control(ipy.LayersControl())
m
ipywidgets.VBox([m, label_name, label_parks, label_buff])

Dropdown(description='Radius within Parks', options=('Quarter Mile Buffer', 'Half Mile Buffer', 'Varied Buffer…